In [1]:
def plot_feature_importances(rf, cols, model_dir='.'):
    importances = pd.DataFrame()
    importances.loc[:, 'importances'] = rf.feature_importances_
    importances.loc[:, 'features'] = cols
    importances.sort_values('importances', inplace=True)
    f, a = plt.subplots()
    importances.plot(ax=a, kind='bar', x='features', y='importances')
    plt.gcf().subplots_adjust(bottom=0.3)

In [2]:
from src.dirs import MODELHOME, DATAHOME
from src.ml import to_dense, TargetEncoding
from src.io import load_artifacts

from joblib import load

rf = load_artifacts(MODELHOME / 'final-rf')
nb = load_artifacts(MODELHOME / 'final-naive')

binner = load(DATAHOME / 'processed' / 'binner.joblib')
rf.keys()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/adam/hacking-medium-headlines/data/processed/binner.joblib'

## prediction accuracy by site

In [ ]:
binner.get_params()

In [ ]:
edge[2]

In [ ]:
edges = binner.bin_edges_

for label, edge in enumerate(edges[0].flatten().tolist()):
    print(label, edge)

In [ ]:
pipe_tr = rf['pipe-tr']
x_ho = rf['x_ho']
y_ho = rf['y_ho']

pred_ho = pipe_tr.predict(x_ho).reshape(-1, 1)
errors = abs(y_ho - pred_ho)
x_ho.loc[:, 'abs-error'] = errors.values
x_ho.loc[:, 'prediction'] = pred_ho

In [ ]:
x_ho.sort_values('abs-error', ascending=False).head(16)

## Group Errors by Site

In [ ]:
site_err = x_ho.groupby('site_id').agg({'abs-error': 'mean', 'binned-class': 'mean'}).sort_values('abs-error')
site_err

In [ ]:
site_err.plot(x='binned-class', y='abs-error', kind='scatter')